In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/nlp"

Mounted at /content/drive
/content/drive/MyDrive/nlp


In [ ]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=550738 sha256=8472379b5cabfb84be375de75b6bf2673f5766abbb5de7e684dd7a50fe4366b1
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [ ]:
import os
from argparse import Namespace
import collections
import nltk.data
import numpy as np
import pandas as pd
import re
import string
import tqdm
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile
import gensim
import torch
import torch.nn as nn
import tqdm
from annoy import AnnoyIndex
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
args = Namespace(
    seed = 1234,
    embedding_size = 300,
    embedding_path = 'glove.6B.300d.txt',
    test_size = 0.2,
    cuda = True,
    device = "cpu",
    rnn_hidden_size = 64,
    num_epochs=100,
    learning_rate=1e-3,
    batch_size=32,
    early_stopping_criteria=5,
    model_state_file="model2.pth",
    inference_file = "inference_ver2.csv"
)

# preprocessing 1. 특수문자 없애기

In [ ]:
inference_data = pd.read_csv("test_for_inference.csv")
train_data = pd.read_csv("train.csv")

In [ ]:
train_tokenize = train_data
train_tokenize['comment'] = train_tokenize['comment'].apply(word_tokenize)

inference_tokenize = inference_data
inference_tokenize['comment'] = inference_tokenize['comment'].apply(word_tokenize)

In [ ]:
def preprocess_text(text):
    text = ' '.join(word.lower() for word in text)
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

train_tokenize['comment'] = train_tokenize['comment'].apply(preprocess_text)
inference_tokenize['comment'] = inference_tokenize['comment'].apply(preprocess_text)

In [ ]:
train_tokenize.to_csv('train_tokenize.csv', index=False)
inference_tokenize.to_csv('inference_tokenize.csv', index=False)

# preprocessing 2. embedding 300 (option dim : 50, 100, 200, 300)

In [ ]:
train_tokenize = pd.read_csv("train_tokenize.csv")
inference_tokenize = pd.read_csv("inference_tokenize.csv")

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!mkdir -p data/glove
!mv glove.6B.100d.txt data/glove

--2023-12-04 09:31:37--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-12-04 09:31:37--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-12-04 09:31:38--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
class PreTrainedEmbeddings(object):
    """ 사전 훈련된 단어 벡터 사용을 위한 래퍼 클래스 """
    def __init__(self, word_to_index, word_vectors):
        """
        매개변수:
            word_to_index (dict): 단어에서 정수로 매핑
            word_vectors (numpy 배열의 리스트)
        """
        self.word_to_index = word_to_index
        self.word_vectors = word_vectors
        self.index_to_word = {v: k for k, v in self.word_to_index.items()}

        self.index = AnnoyIndex(len(word_vectors[0]), metric='euclidean')
        print("인덱스 만드는 중!")
        for _, i in self.word_to_index.items():
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
        print("완료!")

    @classmethod
    def from_embeddings_file(cls, embedding_file):
        """사전 훈련된 벡터 파일에서 객체를 만듭니다.

        벡터 파일은 다음과 같은 포맷입니다:
            word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
            word1 x1_0 x1_1 x1_2 x1_3 ... x1_N

        매개변수:
            embedding_file (str): 파일 위치
        반환값:
            PretrainedEmbeddings의 인스턴스
        """
        word_to_index = {}
        word_vectors = []

        with open(embedding_file) as fp:
            for line in fp.readlines():
                line = line.split(" ")
                word = line[0]
                vec = np.array([float(x) for x in line[1:]])

                word_to_index[word] = len(word_to_index)
                word_vectors.append(vec)

        return cls(word_to_index, word_vectors)

    def get_embedding(self, word):
        """
        매개변수:
            word (str)
        반환값
            임베딩 (numpy.ndarray)
        """
        if word not in self.word_to_index:
            return np.zeros(len(self.word_vectors[0]))
        return self.word_vectors[self.word_to_index[word]]


In [ ]:
embeddings = PreTrainedEmbeddings.from_embeddings_file(args.embedding_path)

인덱스 만드는 중!
완료!


In [ ]:
train_tokenize.head()

,comment,toxicity
0,i am testing . . . .,0
1,honestly you need to crawl under a rock and di...,1
2,the russian idiom to let the he goat into the...,0
3,created and dominated hamilton created and do...,0
4,this one and this one . . . .,0


In [ ]:
inference_tokenize.head()

,comment
0,"vandalism , or just bad writing ? so , i noti..."
1,congratulations on getting m rio de andrade to...
2,link canvassing talk
3,wp overlinking policy on wikipedia generally ...
4,nomination i am officially nominating tenpound...


In [ ]:
n,_ = train_tokenize.shape
train_vector=np.zeros((n,args.embedding_size))

for i in range(n):
  string = train_tokenize.iloc[i,0].split(" ")
  embedding_vetor = np.zeros((len(string),args.embedding_size))
  for j in range(len(string)):
    embedding_vetor[j] = embeddings.get_embedding(string[j])
  train_vector[i] = np.sum(embedding_vetor, axis=0, keepdims=True)

In [ ]:
n,_ = inference_tokenize.shape
inference_vector=np.zeros((n,args.embedding_size))

for i in range(n):
  string = inference_tokenize.iloc[i,0].split(" ")
  embedding_vetor = np.zeros((len(string),args.embedding_size))
  for j in range(len(string)):
    embedding_vetor[j] = embeddings.get_embedding(string[j])
  inference_vector[i] = np.sum(embedding_vetor, axis=0, keepdims=True)

In [ ]:
vec_df = pd.DataFrame(train_vector, columns=[f'vec_{i+1}' for i in range(args.embedding_size)])
vec_inference_df = pd.DataFrame(inference_vector, columns=[f'vec_{i+1}' for i in range(args.embedding_size)])

In [ ]:
vec_df["toxicity"] = train_tokenize["toxicity"]

In [ ]:
vec_df.head()

,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,vec_10,...,vec_292,vec_293,vec_294,vec_295,vec_296,vec_297,vec_298,vec_299,vec_300,toxicity
0,-0.044010,0.638130,0.643555,-1.644222,0.576932,0.293264,-0.381603,0.970170,1.373215,-13.156800,...,-2.055470,-0.937030,0.929410,0.525642,0.294999,0.430040,-1.994840,0.812894,1.440870,0
1,-2.137932,0.457858,-0.935301,-1.004755,-2.222339,0.224438,0.053761,0.558369,1.347558,-16.109180,...,-1.722440,-1.344369,-0.650444,2.022343,-2.065591,0.373522,0.127907,-0.660130,1.145126,1
2,-3.603719,2.812924,-2.325180,-7.742927,-4.067313,-0.398389,-6.787577,0.908065,1.007097,-52.587893,...,-7.019794,1.310713,1.058308,6.047098,9.600655,1.620984,-3.434556,-1.861499,6.886566,0
3,-6.917234,9.243579,1.358257,-12.631775,-2.173060,-2.824434,-0.722976,12.613016,6.047556,-133.755179,...,-17.904944,-0.750353,2.815478,7.256548,17.703225,5.426741,-24.044911,-11.986289,8.735168,0
4,-1.607754,1.133348,0.037895,-1.364650,-0.470134,1.477630,-0.889407,0.576968,1.331120,-15.145800,...,-2.188330,-0.966592,1.543250,1.208514,2.231130,-0.667648,-3.128410,-1.011861,0.501867,0


In [ ]:
vec_inference_df.head()

,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,vec_10,...,vec_291,vec_292,vec_293,vec_294,vec_295,vec_296,vec_297,vec_298,vec_299,vec_300
0,-11.543046,7.965737,-1.220547,-11.465494,-1.940666,1.397912,-7.553515,4.476777,5.346560,-157.389780,...,1.245256,-18.864546,3.965854,8.873872,22.611886,2.548543,-1.086786,-17.996037,-9.190823,5.994960
1,0.578331,0.999319,-0.372997,-0.445800,-0.797562,0.163404,-1.646707,-0.257588,0.118189,-11.963120,...,-2.177730,0.558536,0.170104,-2.252974,1.858840,0.661618,-1.089749,-3.538523,-0.754801,-0.658054
2,-0.392184,-0.281408,0.362548,-0.630010,-0.281298,-0.267850,0.263196,-0.926408,1.317740,-2.987400,...,-0.539130,-0.396760,-0.629654,0.477318,-0.927610,-0.792740,-0.213481,-0.229879,-0.246291,0.808321
3,-51.244999,23.590788,9.651939,-45.018806,-19.155644,16.064427,-15.675763,31.112921,16.385769,-479.130398,...,2.991623,-45.899215,-21.594162,15.751244,33.991960,60.627161,-1.568667,-75.293741,-3.758410,-0.762510
4,-1.341636,0.494394,0.058070,-1.274500,-1.150018,-0.042197,0.324855,-0.154269,1.891121,-18.990340,...,-0.463425,-2.637908,0.178942,-0.267450,1.041334,-0.246789,-0.499579,-1.911536,-0.624130,3.115174


In [ ]:
vec_df.to_csv('train_emb_300.csv', index=False)
vec_inference_df.to_csv('inference_emb_300.csv', index=False)

# preprocessing 3. label 비율에 맞춰 data split

In [ ]:
train_emb = pd.read_csv("train_emb_300.csv")
inference_emb = pd.read_csv("inference_emb_300.csv")

In [ ]:
train_emb.head()

,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,vec_10,...,vec_292,vec_293,vec_294,vec_295,vec_296,vec_297,vec_298,vec_299,vec_300,toxicity
0,-0.044010,0.638130,0.643555,-1.644222,0.576932,0.293264,-0.381603,0.970170,1.373215,-13.156800,...,-2.055470,-0.937030,0.929410,0.525642,0.294999,0.430040,-1.994840,0.812894,1.440870,0
1,-2.137932,0.457858,-0.935301,-1.004755,-2.222339,0.224438,0.053761,0.558369,1.347558,-16.109180,...,-1.722440,-1.344369,-0.650444,2.022343,-2.065591,0.373522,0.127907,-0.660130,1.145126,1
2,-3.603719,2.812924,-2.325180,-7.742927,-4.067313,-0.398389,-6.787577,0.908065,1.007097,-52.587893,...,-7.019794,1.310713,1.058308,6.047098,9.600655,1.620984,-3.434556,-1.861499,6.886566,0
3,-6.917234,9.243579,1.358257,-12.631775,-2.173060,-2.824434,-0.722976,12.613016,6.047556,-133.755179,...,-17.904944,-0.750353,2.815478,7.256548,17.703225,5.426741,-24.044911,-11.986289,8.735168,0
4,-1.607754,1.133348,0.037895,-1.364650,-0.470134,1.477630,-0.889407,0.576968,1.331120,-15.145800,...,-2.188330,-0.966592,1.543250,1.208514,2.231130,-0.667648,-3.128410,-1.011861,0.501867,0


In [ ]:
inference_emb.head()

,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,vec_10,...,vec_291,vec_292,vec_293,vec_294,vec_295,vec_296,vec_297,vec_298,vec_299,vec_300
0,-11.543046,7.965737,-1.220547,-11.465494,-1.940666,1.397912,-7.553515,4.476777,5.346560,-157.389780,...,1.245256,-18.864546,3.965854,8.873872,22.611886,2.548543,-1.086786,-17.996037,-9.190823,5.994960
1,0.578331,0.999319,-0.372997,-0.445800,-0.797562,0.163404,-1.646707,-0.257588,0.118189,-11.963120,...,-2.177730,0.558536,0.170104,-2.252974,1.858840,0.661618,-1.089749,-3.538523,-0.754801,-0.658054
2,-0.392184,-0.281408,0.362548,-0.630010,-0.281298,-0.267850,0.263196,-0.926408,1.317740,-2.987400,...,-0.539130,-0.396760,-0.629654,0.477318,-0.927610,-0.792740,-0.213481,-0.229879,-0.246291,0.808321
3,-51.244999,23.590788,9.651939,-45.018806,-19.155644,16.064427,-15.675763,31.112921,16.385769,-479.130398,...,2.991623,-45.899215,-21.594162,15.751244,33.991960,60.627161,-1.568667,-75.293741,-3.758410,-0.762510
4,-1.341636,0.494394,0.058070,-1.274500,-1.150018,-0.042197,0.324855,-0.154269,1.891121,-18.990340,...,-0.463425,-2.637908,0.178942,-0.267450,1.041334,-0.246789,-0.499579,-1.911536,-0.624130,3.115174


In [ ]:
train_emb[train_emb["toxicity"]==1].shape

(7647, 301)

In [ ]:
train_emb[train_emb["toxicity"]==0].shape

(72138, 301)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_emb, train_emb['toxicity'], test_size=args.test_size, stratify=train_emb['toxicity'], random_state=args.seed)

In [ ]:
X_train[X_train['toxicity']==1].shape

(6118, 301)

In [ ]:
X_train[X_train['toxicity']==0].shape

(57710, 301)

In [ ]:
X_train = X_train.drop(['toxicity'], axis=1)
X_test = X_test.drop(['toxicity'], axis=1)

# mlp

In [ ]:
def generate_batches(dataset, batch_size, shuffle=False,
                     drop_last=False, device="cpu"):
    """
    파이토치 DataLoader를 감싸고 있는 제너레이터 함수.
    걱 텐서를 지정된 장치로 이동합니다.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
          out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [ ]:
class SurnameDataset(Dataset):
    def __init__(self, X_train,X_test,y_train, y_test,inference_emb):
        """
        매개변수:
            surname_df (pandas.DataFrame): 데이터셋
            vectorizer (SurnameVectorizer): 데이터셋에서 만든 Vectorizer 객체
        """

        self.train_df = X_train
        self.train_size = X_train.shape[0]

        self.val_df = X_test
        self.validation_size = X_test.shape[0]

        self.inference_emb = inference_emb
        self.inference_size = inference_emb.shape[0]



        self._lookup_dict = {'train': (self.train_df,self.train_size,y_train),
                             'val': (self.val_df,self.validation_size, y_test),
                             'inference': (self.inference_emb,self.inference_size,None)}


        self.set_split('train')
        frequencies = [1 for i in range(self.train_size)]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)

    def set_split(self, split="train"):
        self._target_split = split
        self._target_df, self._target_size,_ = self._lookup_dict[split]

    def __getitem__(self, index):
      surname_vector = self._target_df.iloc[index].values  # Convert to numpy array
      if self._target_split == 'inference':
        return {'x_data': torch.tensor(surname_vector, dtype=torch.float32),  # Convert to tensor
                'x_length': len(surname_vector)}
      nationality_index = self._lookup_dict[self._target_split][2].iloc[index]
      nationality_index = torch.tensor(nationality_index, dtype=torch.long)
      vec_length = len(surname_vector)

      return {'x_data': torch.tensor(surname_vector, dtype=torch.float32),  # Convert to tensor
              'y_target': nationality_index,  # Assuming it's an index
              'x_length': vec_length}


    def __len__(self):
        return self._target_size

    def get_num_batches(self, batch_size):
        """배치 크기가 주어지면 데이터셋으로 만들 수 있는 배치 개수를 반환합니다

        매개변수:
            batch_size (int)
        반환값:
            배치 개수
        """
        return len(self) // batch_size

In [ ]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """ 훈련 상태를 업데이트합니다.

    Components:
     - 조기 종료: 과대 적합 방지
     - 모델 체크포인트: 더 나은 모델을 저장합니다

    :param args: 메인 매개변수
    :param model: 훈련할 모델
    :param train_state: 훈련 상태를 담은 딕셔너리
    :returns:
        새로운 훈련 상태
    """

    # 적어도 한 번 모델을 저장합니다
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # 성능이 향상되면 모델을 저장합니다
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # 손실이 나빠지면
        if loss_t >= train_state['early_stopping_best_val']:
            # 조기 종료 단계 업데이트
            train_state['early_stopping_step'] += 1
        # 손실이 감소하면
        else:
            # 최상의 모델 저장
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # 조기 종료 단계 재설정
            train_state['early_stopping_step'] = 0

        # 조기 종료 여부 확인
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

def compute_f1score(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    y_true = y_target.cpu().numpy()
    y_pred = y_pred_indices.cpu().numpy()

    f1 = f1_score(y_true, y_pred, average='weighted')

    return f1

In [ ]:
class SurnameClassifier(nn.Module):
    """ 성씨 분류를 위한 다층 퍼셉트론 """
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        매개변수:
            input_dim (int): 입력 벡터 크기
            hidden_dim (int): 첫 번째 Linear 층의 출력 크기
            output_dim (int): 두 번째 Linear 층의 출력 크기
        """
        super(SurnameClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim,hidden_dim)
        self.fc2 = nn.Linear(hidden_dim,output_dim)

    def forward(self, x_in, apply_softmax=True):
        """MLP의 정방향 계산

        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스-엔트로피 손실을 사용하려면 False로 지정해야 합니다.
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim)입니다.
        """
        intermediate = F.relu(self.fc1(x_in))
        prediction_vector = self.fc2(intermediate)

        prediction_vector = F.softmax(prediction_vector,dim=1)

        return prediction_vector

In [ ]:
# CUDA 체크
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")

classifier = SurnameClassifier(input_dim=args.embedding_size,
                               output_dim=2,
                               hidden_dim=args.rnn_hidden_size)
classifier = classifier.to(args.device)

dataset = SurnameDataset(X_train, X_test, y_train, y_test, inference_emb)
dataset.class_weights = dataset.class_weights.to(args.device)

# 손실 함수와 옵티마이저
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.5,
                                                 patience=1)

train_state = make_train_state(args)

In [ ]:
epoch_bar = tqdm.notebook.tqdm(desc='training routine',
                               total=args.num_epochs,
                               position=0)

dataset.set_split('train')
train_bar = tqdm.notebook.tqdm(desc='split=train',
                               total=dataset.get_num_batches(args.batch_size),
                               position=1,
                               leave=True)
dataset.set_split('val')
val_bar = tqdm.notebook.tqdm(desc='split=val',
                             total=dataset.get_num_batches(args.batch_size),
                             position=1,
                             leave=True)


for epoch_index in range(args.num_epochs):
    train_state['epoch_index'] = epoch_index

    # 훈련 세트에 대한 순회

    # 훈련 세트와 배치 제너레이터 준비, 손실과 정확도를 0으로 설정
    dataset.set_split('train')
    batch_generator = generate_batches(dataset,
                                       batch_size=args.batch_size,
                                       device=args.device)
    running_loss = 0.0
    running_acc = 0.0
    classifier.train()

    for batch_index, batch_dict in enumerate(batch_generator):
        # 훈련 과정은 5단계로 이루어집니다

        # --------------------------------------
        # 단계 1. 그레이디언트를 0으로 초기화합니다
        optimizer.zero_grad()

        # 단계 2. 출력을 계산합니다
        y_pred = classifier(batch_dict['x_data'])

        # 단계 3. 손실을 계산합니다
        loss = loss_func(y_pred, batch_dict['y_target'])
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

        # 단계 4. 손실을 사용해 그레이디언트를 계산합니다
        loss.backward()

        # 단계 5. 옵티마이저로 가중치를 업데이트합니다
        optimizer.step()
        # -----------------------------------------

        # 정확도를 계산합니다
        acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_t - running_acc) / (batch_index + 1)

        # 진행 바 업데이트
        train_bar.set_postfix(loss=running_loss, acc=running_acc,
                        epoch=epoch_index)
        train_bar.update()

    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)

    # 검증 세트에 대한 순회

    # 검증 세트와 배치 제너레이터 준비, 손실과 정확도를 0으로 설정
    dataset.set_split('val')
    batch_generator = generate_batches(dataset,
                                       batch_size=args.batch_size,
                                       device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):

        # 단계 1. 출력을 계산합니다
        y_pred =  classifier(batch_dict['x_data'])

        # 단계 2. 손실을 계산합니다
        loss = loss_func(y_pred, batch_dict['y_target'])
        loss_t = loss.to("cpu").item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

        # 단계 3. 정확도를 계산합니다
        acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_t - running_acc) / (batch_index + 1)
        val_bar.set_postfix(loss=running_loss, acc=running_acc,
                        epoch=epoch_index)
        val_bar.update()

    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)

    train_state = update_train_state(args=args, model=classifier,
                                     train_state=train_state)

    scheduler.step(train_state['val_loss'][-1])

    if train_state['stop_early']:
        break

    train_bar.n = 0
    val_bar.n = 0
    epoch_bar.update()

training routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/1994 [00:00<?, ?it/s]

split=val:   0%|          | 0/498 [00:00<?, ?it/s]

# 검증

In [ ]:
# 가장 좋은 모델을 사용해 테스트 세트의 손실과 정확도를 계산합니다
classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
loss_func = nn.CrossEntropyLoss()

dataset.set_split('val')
batch_generator = generate_batches(dataset,
                                   batch_size=args.batch_size,
                                   device=args.device)
running_loss = 0.
running_acc = 0.
running_f1 = 0
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # 출력을 계산합니다
    y_pred =  classifier(batch_dict['x_data'])

    # 손실을 계산합니다
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # 정확도를 계산합니다
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    f1_t = compute_f1score( y_pred,batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)
    running_f1 += (f1_t - running_f1) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc


print("검증 손실: {}".format(train_state['test_loss']))
print("검증 정확도: {}".format(train_state['test_acc']))
print("검증 f1-score: {}".format(running_f1))

검증 손실: 0.361930893513865
검증 정확도: 94.98669958965547
검증 f1-score: 0.9451090380334451


# inference

In [ ]:
# 가장 좋은 모델을 사용해 테스트 세트의 손실과 정확도를 계산합니다
classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
loss_func = nn.CrossEntropyLoss()

dataset.set_split('inference')
batch_generator = generate_batches(dataset,
                                   batch_size=args.batch_size,
                                   device=args.device)
classifier.eval()

pred_tensor = torch.tensor([]).to(args.device)
for batch_index, batch_dict in enumerate(batch_generator):
    y_pred =  classifier(batch_dict['x_data'])

    pred_tensor= torch.cat((pred_tensor, y_pred), dim=0)
pred_tensor = pred_tensor.cpu().detach()

df = pd.DataFrame(columns=['probability', 'pred'])

for prob1, prob2 in pred_tensor:
    prob1 = prob1.item()
    prob2 = prob2.item()
    if prob1 > prob2:
        df = df.append({'probability': prob1, 'pred': 0}, ignore_index=True)
    else:
        df = df.append({'probability': prob2, 'pred': 1}, ignore_index=True)
df.to_csv(args.inference_file, index=False)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-41-a54d715aecc1>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'probability': prob1, 'pred': 0}, ignore_index=True)
<ipython-input-41-a54d715aecc1>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'probability': prob1, 'pred': 0}, ignore_index=True)
<ipython-input-41-a54d715aecc1>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'probability': prob2, 'pred': 1}, ignore_index=True)
<ipython-input-41-a54d715aecc1>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'probability': prob1, 'pred': 0}, ignore_index=True)
<ipy

In [ ]:
df[df['pred']==1]

,probability,pred
15,1.000000,1.0
20,1.000000,1.0
51,0.997303,1.0
56,1.000000,1.0
60,1.000000,1.0
...,...,...
31872,1.000000,1.0
31878,1.000000,1.0
31884,0.999994,1.0
31896,0.993914,1.0
